In [1]:
import pandas as pd
from sys import exit

from bulletin.data import cievs, municipios, regionais
from bulletin.commom import normalize

In [2]:
mun_list = municipios.read()
rs_list = regionais.read()

In [3]:
casos = cievs.read_casos()

obitos = cievs.read_obitos()

print(f"\ncasos: {len(casos)}\tobitos: {len(obitos)}")

C:\SESA\sesa-covid19\bulletin\tmp\cievs.xlsx:Casos confirmados loaded
C:\SESA\sesa-covid19\bulletin\tmp\cievs_casos.pkl saved
C:\SESA\sesa-covid19\bulletin\tmp\cievs.xlsx:Obitos loaded
C:\SESA\sesa-covid19\bulletin\tmp\cievs_obitos.pkl saved

casos: 221208	obitos: 5392


In [4]:
obitos

,hash,dt_obt
0,24ee2f0066ba7b592fa53e8916636d0d,2020-03-25
1,b132a42f8163fcc4dc4fcf3af4dbcd62,2020-03-26
2,256a49075d9a296818394cc0608e6193,2020-03-30
3,f4225b8a7b15283adaa58a003fc39c3f,2020-03-31
4,1b82e3083c146b5e6c7312237b80a9c8,2020-04-03
...,...,...
5397,e5c8bdeb233d5d1a295c49e062e42184,2020-10-22
5398,9e4de02b336665498f54c726de07d575,2020-10-13
5399,e6d1e3a7989303fe1005e553e6a648cc,2020-11-07
5400,e19c407663d7787385909e7eaa94e6a2,2020-11-08


In [5]:
casos['dt_com'] = casos['dt_com'].apply(cievs.add_dt_com)

In [6]:
casos = pd.merge(left=casos, right=mun_list, how='left', left_on='ibge',right_on='ibge')
print(casos.shape)
casos = pd.merge(left=casos, right=rs_list, how='left', left_on='ibge',right_on='ibge')
print(casos.shape)
casos = pd.merge(left=casos, right=obitos, how='left', on='hash')
print(casos.shape)

(221208, 12)
(221208, 16)
(221210, 17)


In [7]:
casos

,hash,sexo,idade,ibge,mun_resid,mun_atend,laboratorio,dt_com,dt_diag,uf,mun_nome,populacao,reg,nm_reg,nm_macro,cep,dt_obt
0,3a321250efbd7bd85552fb70bcde1099,f,47,410550,cianorte,cianorte,lacen,2020-03-12,2020-03-12,PR,Cianorte,83816.0,13.0,Cianorte,Noroeste,87200000,NaN
1,aa4754dee32706fdb181e93bfa24cbb9,m,43,410690,curitiba,curitiba,lacen,2020-03-12,2020-03-12,PR,Curitiba,1948626.0,2.0,Metropolitana,Leste,80001970,NaN
2,a51b10372da4193df0478ba884ff5044,m,15,410690,curitiba,curitiba,lacen,2020-03-12,2020-03-12,PR,Curitiba,1948626.0,2.0,Metropolitana,Leste,80001970,NaN
3,039e6accbff1ee57015a119cc3201c25,f,26,410690,curitiba,curitiba,genoprimer,2020-03-16,2020-03-16,PR,Curitiba,1948626.0,2.0,Metropolitana,Leste,80001970,NaN
4,c13b8a002b7178ecbad2177ed71f74a7,f,26,410690,curitiba,curitiba,genoprimer,2020-03-18,2020-03-18,PR,Curitiba,1948626.0,2.0,Metropolitana,Leste,80001970,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221205,853a0efe6dced7b66e9eca79a2bf7eda,m,22,410180,araucaria,araucaria,covid-19_biologia_molecular,2020-11-09,2020-11-08,PR,Araucária,146214.0,2.0,Metropolitana,Leste,83700970,NaN
221206,587ccb94021b361ab77c34875a8be1b8,m,24,411520,maringa,maringa,covid-19_biologia_molecular,2020-11-09,2020-11-07,PR,Maringá,430157.0,15.0,Maringa,Noroeste,87001970,NaN
221207,a48bda562faea2fc342e895776fbdfb3,f,11,411370,londrina,londrina,covid-19_biologia_molecular,2020-11-09,2020-11-06,PR,Londrina,575377.0,17.0,Londrina,Norte,86001970,NaN
221208,39f621d18e8eb2a9295dd625cbfa94e5,f,46,410180,araucaria,araucaria,covid-19_biologia_molecular,2020-11-09,2020-11-08,PR,Araucária,146214.0,2.0,Metropolitana,Leste,83700970,NaN


In [8]:
casos_pr = casos.loc[casos['ibge']!=-1]
casos_fora = casos.loc[casos['ibge']==-1]
print(f"Casos\nTotal:\t{len(casos)}\nParaná:\t{len(casos_pr)}\nFora:\t{len(casos_fora)}")

Casos
Total:	221210
Paraná:	218929
Fora:	2281


In [9]:
obitos = casos.loc[casos['dt_obt'].notnull()]
obitos_pr = casos_pr.loc[casos_pr['dt_obt'].notnull()]
obitos_fora = casos_fora.loc[casos_fora['dt_obt'].notnull()]
print(f"Obitos\nTotal:\t{len(obitos)}\nParaná:\t{len(obitos_pr)}\nFora:\t{len(obitos_fora)}")

Obitos
Total:	5381
Paraná:	5340
Fora:	41
